In [1]:
# Imports
import pandas as pd
import random
import numpy as np
from gurobipy import Model, GRB, quicksum

In [2]:
# # Function to generate unique random animal names
# def generate_team_names(num_teams):
#     animals = ["Eagles", "Lions", "Wolves", "Dolphins", "Falcons", "Tigers", "Owls", "Hawks", "Cougars", "Sharks", "Panthers", "Bears"]
#     random.shuffle(animals)
#     return animals[:num_teams]

# # Function to create the fantasy football league schedule for the first 11 weeks
# def create_first_11_weeks_schedule(num_teams):
#     team_names = generate_team_names(num_teams)
#     schedule = []

#     # Generate the first 11 weeks where each team plays each other team once
#     for week in range(1, num_teams):
#         matchups = []

#         for i in range(num_teams // 2):
#             home_team = team_names[i]
#             away_team = team_names[num_teams - 1 - i]

#             if week % 2 == 0:  # Switch home and away every week
#                 home_team, away_team = away_team, home_team

#             matchups.append((week, home_team, away_team))

#         schedule.extend(matchups)
#         team_names = [team_names[-1]] + team_names[:-1]  # Rotate teams for the next week

#     return schedule

# # Function to extend the schedule for weeks 12-14
# def extend_schedule_for_weeks_12_14(schedule):
#     # Copy the matchups for weeks 1-3 and switch home and away for weeks 12-14
#     copied_matchups = schedule[:num_teams - 1]
#     copied_matchups = [(week + 11, home_team, away_team) for week, home_team, away_team in copied_matchups]
#     schedule.extend(copied_matchups)

# # Create the fantasy football league schedule for the first 11 weeks
# num_teams = 12
# first_11_weeks_schedule = create_first_11_weeks_schedule(num_teams)

# # Extend the schedule for weeks 12-14
# extend_schedule_for_weeks_12_14(first_11_weeks_schedule)

# # Create a pandas DataFrame with the schedule data
# columns = ['Week', 'Team', 'Opponent']
# season_schedule = pd.DataFrame(first_11_weeks_schedule, columns=columns)

# Read the CSV file into a DataFrame
season_schedule = pd.read_csv('schedule_teams.csv')
season_schedule.sort_values(by=['Week', 'Team'], inplace=True)

# Display the schedule
print(season_schedule)

    Week     Team  Opponent
3      1    Bears     Lions
5      1  Falcons    Eagles
4      1    Hawks   Cougars
1      1     Owls  Panthers
2      1   Sharks    Wolves
..   ...      ...       ...
81    14   Eagles      Owls
82    14  Falcons    Sharks
83    14    Hawks     Bears
79    14    Lions  Dolphins
78    14   Wolves  Panthers

[84 rows x 3 columns]


In [3]:
# Create a list of the teams
def generate_team_names(num_teams):
    animals = ["Eagles", "Lions", "Wolves", "Dolphins", "Falcons", "Tigers", "Owls", "Hawks", "Cougars", "Sharks", "Panthers", "Bears"]
    random.shuffle(animals)
    return animals[:num_teams]

# Generate schedule for a specific team
def generate_team_schedule(teams, target_team):
    rotating_teams = teams.copy()
    rotating_teams.remove(target_team)
    random.shuffle(rotating_teams)

    schedule = {target_team: {}}
    for week in range(1, 15):  # Assuming a 14-week season
        matchups = list(zip([target_team] + rotating_teams[:len(rotating_teams)//2], rotating_teams[len(rotating_teams)//2:]))
        opponent = None

        for team1, team2 in matchups:
            if team1 == target_team:
                opponent = team2
                break

            if team2 == target_team:
                opponent = team1
                break

        schedule[target_team][week] = opponent
        rotating_teams = rotating_teams[-1:] + rotating_teams[:-1]  # Rotate the teams

    return schedule

# Generate animal team names for 12 teams
teams = generate_team_names(12)

# Choose the team for which you want to generate the schedule (e.g., "Bears")
target_team = "Bears"

# Generate the schedule for the specified team
team_schedule = generate_team_schedule(teams, target_team)

# Print the generated animal team names
print("Animal Team Names:", teams)

# Print the generated schedule for the specified team
print(f"\nGenerated Schedule for {target_team}:")
for week, opponent in team_schedule[target_team].items():
    print(f"Week {week}: {opponent}")


Animal Team Names: ['Cougars', 'Tigers', 'Owls', 'Panthers', 'Lions', 'Bears', 'Wolves', 'Sharks', 'Hawks', 'Eagles', 'Falcons', 'Dolphins']

Generated Schedule for Bears:
Week 1: Eagles
Week 2: Hawks
Week 3: Sharks
Week 4: Falcons
Week 5: Panthers
Week 6: Cougars
Week 7: Dolphins
Week 8: Wolves
Week 9: Tigers
Week 10: Owls
Week 11: Lions
Week 12: Eagles
Week 13: Hawks
Week 14: Sharks


In [4]:
# Generate draft order
def generate_team_names(num_teams):
    animals = ["Eagles", "Lions", "Wolves", "Dolphins", "Falcons", "Tigers", "Owls", "Hawks", "Cougars", "Sharks", "Panthers", "Bears"]
    random.shuffle(animals)
    return animals[:num_teams]

# Generate animal team names
team_names = generate_team_names(10)

# Set the initial draft order
initial_draft_order = team_names

# Generate the Snake-Draft order for all rounds
draft_order = []
for r in range(16):
    if r % 2 == 0:
        draft_order.append(initial_draft_order)
    else:
        draft_order.append(initial_draft_order[::-1])  # Reverse the order

# Display the generated team names and draft order
print("Team Names:", team_names)
for round_num, order in enumerate(draft_order, 1):
    print(f"Round {round_num} Draft Order:", order)


Team Names: ['Falcons', 'Panthers', 'Owls', 'Bears', 'Wolves', 'Eagles', 'Tigers', 'Dolphins', 'Hawks', 'Lions']
Round 1 Draft Order: ['Falcons', 'Panthers', 'Owls', 'Bears', 'Wolves', 'Eagles', 'Tigers', 'Dolphins', 'Hawks', 'Lions']
Round 2 Draft Order: ['Lions', 'Hawks', 'Dolphins', 'Tigers', 'Eagles', 'Wolves', 'Bears', 'Owls', 'Panthers', 'Falcons']
Round 3 Draft Order: ['Falcons', 'Panthers', 'Owls', 'Bears', 'Wolves', 'Eagles', 'Tigers', 'Dolphins', 'Hawks', 'Lions']
Round 4 Draft Order: ['Lions', 'Hawks', 'Dolphins', 'Tigers', 'Eagles', 'Wolves', 'Bears', 'Owls', 'Panthers', 'Falcons']
Round 5 Draft Order: ['Falcons', 'Panthers', 'Owls', 'Bears', 'Wolves', 'Eagles', 'Tigers', 'Dolphins', 'Hawks', 'Lions']
Round 6 Draft Order: ['Lions', 'Hawks', 'Dolphins', 'Tigers', 'Eagles', 'Wolves', 'Bears', 'Owls', 'Panthers', 'Falcons']
Round 7 Draft Order: ['Falcons', 'Panthers', 'Owls', 'Bears', 'Wolves', 'Eagles', 'Tigers', 'Dolphins', 'Hawks', 'Lions']
Round 8 Draft Order: ['Lions', 'H

In [5]:
# Load prepared data
fantasy_data_player_rf = pd.read_csv('fantasy_data_player_rf.csv')
fantasy_data_player_xgb = pd.read_csv('fantasy_data_player_xgb.csv')
adp_and_vor_ranks = pd.read_csv('adp_and_vor_ranks.csv')
consensus_ranks = pd.read_csv('consensus_ranks.csv')
merged_adp_ranks = pd.read_csv('merged_adp_ranks.csv')
opportunity_cost_avg_ranks = pd.read_csv('opportunity_cost_avg_ranks.csv')
opportunity_cost_ranks = pd.read_csv('opportunity_cost_ranks.csv')

# Adjust sorting of ranks if necessary and keep only necessary columns
adp_and_vor_ranks = adp_and_vor_ranks.sort_values(by=['rank'])
adp_and_vor_ranks = adp_and_vor_ranks[['player_id', 'Player', 'POS', 'position_group', 'adp_rank', 'rank']]
consensus_ranks = consensus_ranks.sort_values(by=['rank'])
consensus_ranks = consensus_ranks[['player_id', 'Player', 'POS', 'position_group', 'rank']]
merged_adp_ranks = merged_adp_ranks.sort_values(by=['rank'])
merged_adp_ranks = merged_adp_ranks[['player_id', 'Player', 'POS', 'position_group', 'rank']]
opportunity_cost_avg_ranks = opportunity_cost_avg_ranks.sort_values(by=['rank'])
opportunity_cost_avg_ranks = opportunity_cost_avg_ranks[['player_id', 'Player', 'POS', 'position_group', 'rank']]
opportunity_cost_ranks = opportunity_cost_ranks.sort_values(by=['rank'])
opportunity_cost_ranks = opportunity_cost_ranks[['player_id', 'Player', 'POS', 'position_group', 'rank']]


In [6]:
# Set Multi-index for Fantasy Player Data
fantasy_data_player_rf.set_index(['player_id', 'week'], inplace=True)

# Generate Matrix of Expected Fantasy Points
projected_points = fantasy_data_player_rf.pivot_table(values='predicted_fantasy_points', index=['player_id'], columns=['week'], fill_value=0)

# Reset Index for Fantasy Player Data
fantasy_data_player_rf = fantasy_data_player_rf.reset_index()

# Transform to Long-format DataFrame
full_projected_points = projected_points.reset_index().melt(id_vars='player_id', var_name='week', value_name='points')
full_projected_points['week'] = full_projected_points['week'].astype(int)  # Ensure 'week' is an integer

# Create a dictionary from the long-format DataFrame
projected_points_dict = {(row['player_id'], row['week']): row['points'] for index, row in full_projected_points.iterrows()}

# # Display the dictionary or DataFrame
# print(projected_points_dict)


In [7]:
# List of players, teams, and weeks
players = fantasy_data_player_rf['player_id'].unique().tolist()
teams = season_schedule['Team'].unique().tolist()
weeks = list(range(1, 15))
position = dict(zip(fantasy_data_player_rf['player_id'], fantasy_data_player_rf['position_group']))

# Assuming each team selects 16 players
num_players_per_team = 16
rounds = list(range(1, num_players_per_team + 1))

# Create a dictionary to store the rank for each player over the entire season
season_ranking = {row['player_id']: row['rank'] for index, row in adp_and_vor_ranks.iterrows()}

# Initialize the model
m = Model("Fantasy_Football_Optimization")

# l[p][k][t] is 1, if player p is selected and played in week k by team t
l = m.addVars(players, weeks, teams, vtype=GRB.BINARY, name="played")

# s[p][t] is 1, if player p is selected by team t for the entire season
s = m.addVars(players, teams, vtype=GRB.BINARY, name="selected_by")

# d[t][r] is 1, if team t selects a player in round r
d = m.addVars(teams, rounds, vtype=GRB.BINARY, name="drafted")

# w[t][k] is 1, if team t wins in week k
w = m.addVars(teams, weeks, vtype=GRB.BINARY, name="wins")

# Objective function: Maximize the number of wins for Team Bears
m.setObjective(quicksum(w['Bears', k] for k in weeks), GRB.MAXIMIZE)

# Schedule constraints
for k in weeks:
    opponent = team_schedule['Bears'][k]
    m.addConstr(quicksum(l[p, k, 'Bears'] * projected_points_dict[(p, k)] for p in players) -
                quicksum(l[p, k, opponent] * projected_points_dict[(p, k)] for p in players) >=
                w['Bears', k] - (1 - w[opponent, k]))

for t in teams:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for p in players) == 9)

# Add position constraints
for t in teams:
    for k in weeks:
        # Quarterback Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'QB') == 1)

        # Flex Constraint
        m.addConstr(
            quicksum(l[p, k, t] for p in players if position[p] in ['WR', 'RB']) +
            quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 3
        )

        # Running Back Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'RB') >= 2)

        # Wide Receiver Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'WR') >= 2)

        # Tight End Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 1)

        # Kicker Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'K') == 1)

        # Defense/Special Teams Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'DST') == 1)



for p in players:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for t in teams) <= 1)

for p in players:
    for t in teams:
        m.addConstr(s[p, t] >= quicksum(l[p, k, t] for k in weeks) / len(weeks))

for p in players:
    m.addConstr(quicksum(s[p, t] for t in teams) <= 1)

for t in teams:
    if t != 'Bears':
        m.addConstr(quicksum(s[p, t] * season_ranking[p] for p in players if p in season_ranking and (p, t) in s) <= len(players))

for t in teams:
    m.addConstr(quicksum(s[p, t] for p in players) <= num_players_per_team)

for r in rounds:
    for t in draft_order[r-1]:
        m.addConstr(d[t, r] == 1)

# Solve the problem
m.optimize()

# Output the results
for v in m.getVars():
    if v.x > 0:
        print('%s : %g' % (v.varName, v.x))

# Extract selected players for each team and week
selected_players_adp_vor = [(p, k, t) for p in players for k in weeks for t in teams if l[p, k, t].x == 1]

# Convert the list to a DataFrame for better presentation
weekly_lineups_adp_vor = pd.DataFrame(selected_players_adp_vor, columns=['Player ID', 'Week', 'Team'])

weekly_lineups_adp_vor


Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-06
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 19658 rows, 121140 columns and 577838 nonzeros
Model fingerprint: 0x8114b84d
Variable types: 0 continuous, 121140 integer (121140 binary)
Coefficient statistics:
  Matrix range     [7e-02, 4e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]
Presolve removed 328 rows and 346 columns
Presolve time: 3.16s
Presolved: 19330 rows, 120794 columns, 464916 nonzeros
Variable types: 0 continuous, 120794 integer (120794 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -0.

,Player ID,Week,Team
0,00-0023459,1,Eagles
1,00-0023459,2,Eagles
2,00-0023459,7,Eagles
3,00-0023459,8,Eagles
4,00-0023459,9,Eagles
...,...,...,...
1507,00-0090021,10,Lions
1508,00-0090021,11,Lions
1509,00-0090021,12,Lions
1510,00-0090021,13,Lions


In [8]:
# Displaying weekly fantasy points scored by different teams
weekly_points = pd.merge(weekly_lineups_adp_vor, fantasy_data_player_rf[['player_id', 'week', 'fantasy_points_ppr']], 
                      left_on=['Player ID', 'Week'], 
                      right_on=['player_id', 'week'], 
                      how='left')

weekly_points = weekly_points.groupby(['Team', 'Week']).agg({'fantasy_points_ppr': 'sum'}).reset_index()

weekly_points = weekly_points.rename(columns={'fantasy_points_ppr': 'Total Points'})

print(weekly_points[weekly_points['Team'] == 'Bears'])

print(weekly_points[weekly_points['Team'] != 'Bears'])


     Team  Week  Total Points
0   Bears     1         31.30
1   Bears     2         42.76
2   Bears     3         69.58
3   Bears     4         38.04
4   Bears     5         70.38
5   Bears     6         37.78
6   Bears     7         35.88
7   Bears     8         43.96
8   Bears     9         45.14
9   Bears    10         86.22
10  Bears    11         48.22
11  Bears    12         41.30
12  Bears    13         55.90
13  Bears    14         53.22
        Team  Week  Total Points
14   Cougars     1         34.20
15   Cougars     2         31.70
16   Cougars     3         13.10
17   Cougars     4          3.20
18   Cougars     5         21.40
..       ...   ...           ...
163   Wolves    10         38.42
164   Wolves    11         41.96
165   Wolves    12         37.52
166   Wolves    13         14.60
167   Wolves    14         18.00

[154 rows x 3 columns]


In [9]:
# Displaying Bears game results with Result column
games_team = pd.concat({k: pd.Series(v) for k, v in team_schedule.items()}).reset_index()
games_team.columns = ['Team', 'Week', 'Opponent']

match_score = pd.merge(weekly_points, games_team, how='left', on=['Team', 'Week'])

match_score['Opponent Points'] = match_score.apply(
    lambda row: weekly_points[
        (weekly_points['Team'] == row['Opponent']) &
        (weekly_points['Week'] == row['Week'])
    ]['Total Points'].values[0]
    if row['Opponent'] in weekly_points['Team'].values else 0,
    axis=1
)

match_score['Result'] = np.where(match_score['Total Points'] > match_score['Opponent Points'], 'Win', 'Loss')

team_results = match_score[match_score['Team'] == 'Bears']

print(team_results[['Week', 'Team', 'Total Points', 'Opponent', 'Opponent Points', 'Result']])


    Week   Team  Total Points  Opponent  Opponent Points Result
0      1  Bears         31.30    Eagles            16.32    Win
1      2  Bears         42.76     Hawks            13.90    Win
2      3  Bears         69.58    Sharks            49.50    Win
3      4  Bears         38.04   Falcons            11.50    Win
4      5  Bears         70.38  Panthers             2.30    Win
5      6  Bears         37.78   Cougars             0.00    Win
6      7  Bears         35.88  Dolphins            25.30    Win
7      8  Bears         43.96    Wolves            43.36    Win
8      9  Bears         45.14    Tigers            44.60    Win
9     10  Bears         86.22      Owls            54.84    Win
10    11  Bears         48.22     Lions             2.50    Win
11    12  Bears         41.30    Eagles            26.98    Win
12    13  Bears         55.90     Hawks            19.40    Win
13    14  Bears         53.22    Sharks            37.80    Win


In [10]:
# List of players, teams, and weeks
players = fantasy_data_player_rf['player_id'].unique().tolist()
teams = season_schedule['Team'].unique().tolist()
weeks = list(range(1, 15))
position = dict(zip(fantasy_data_player_rf['player_id'], fantasy_data_player_rf['position_group']))

# Assuming each team selects 16 players
num_players_per_team = 16
rounds = list(range(1, num_players_per_team + 1))

# Create a dictionary to store the rank for each player over the entire season
season_ranking = {row['player_id']: row['rank'] for index, row in merged_adp_ranks.iterrows()}

# Initialize the model
m = Model("Fantasy_Football_Optimization")

# l[p][k][t] is 1, if player p is selected and played in week k by team t
l = m.addVars(players, weeks, teams, vtype=GRB.BINARY, name="chosen")

# s[p][t] is 1, if player p is selected by team t for the entire season
s = m.addVars(players, teams, vtype=GRB.BINARY, name="selected_by")

# d[t][r] is 1, if team t selects a player in round r
d = m.addVars(teams, rounds, vtype=GRB.BINARY, name="drafted")

# w[t][k] is 1, if team t wins in week k
w = m.addVars(teams, weeks, vtype=GRB.BINARY, name="wins")

# Objective function: Maximize the number of wins for Manager_1
m.setObjective(quicksum(w['Bears', k] for k in weeks), GRB.MAXIMIZE)

# Schedule constraints
for k in weeks:
    opponent = team_schedule['Bears'][k]
    m.addConstr(quicksum(l[p, k, 'Bears'] * projected_points_dict[(p, k)] for p in players) -
                quicksum(l[p, k, opponent] * projected_points_dict[(p, k)] for p in players) >=
                w['Bears', k] - (1 - w[opponent, k]))

for t in teams:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for p in players) == 9)

# Add position constraints
for t in teams:
    for k in weeks:
        # Quarterback Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'QB') == 1)

        # Flex Constraint
        m.addConstr(
            quicksum(l[p, k, t] for p in players if position[p] in ['WR', 'RB']) +
            quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 3
        )

        # Running Back Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'RB') >= 2)

        # Wide Receiver Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'WR') >= 2)

        # Tight End Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 1)

        # Kicker Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'K') == 1)

        # Defense/Special Teams Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'DST') == 1)

for p in players:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for t in teams) <= 1)

for p in players:
    for t in teams:
        m.addConstr(s[p, t] >= quicksum(l[p, k, t] for k in weeks) / len(weeks))

for p in players:
    m.addConstr(quicksum(s[p, t] for t in teams) <= 1)

for t in teams:
    if t != 'Bears':
        m.addConstr(quicksum(s[p, t] * season_ranking[p] for p in players if p in season_ranking and (p, t) in s) <= len(players))

for t in teams:
    m.addConstr(quicksum(s[p, t] for p in players) <= num_players_per_team)

for r in rounds:
    for t in draft_order[r-1]:
        m.addConstr(d[t, r] == 1)

# Solve the problem
m.optimize()

# Output the results
for v in m.getVars():
    if v.x > 0:
        print('%s : %g' % (v.varName, v.x))

# Extract selected players for each team and week
selected_players_merged_adp = [(p, k, t) for p in players for k in weeks for t in teams if l[p, k, t].x == 1]

# Convert the list to a DataFrame for better presentation
weekly_lineups_merged_adp = pd.DataFrame(selected_players_merged_adp, columns=['Player ID', 'Week', 'Team'])

weekly_lineups_merged_adp


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 19658 rows, 121140 columns and 577838 nonzeros
Model fingerprint: 0x52167483
Variable types: 0 continuous, 121140 integer (121140 binary)
Coefficient statistics:
  Matrix range     [7e-02, 4e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]
Presolve removed 328 rows and 346 columns
Presolve time: 2.37s
Presolved: 19330 rows, 120794 columns, 464916 nonzeros
Variable types: 0 continuous, 120794 integer (120794 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    2728   -0.0000000e+00   4.335714e+02   7.156541e+10      5s
Concurrent spin time: 0.01s

Solved with 

,Player ID,Week,Team
0,00-0019596,1,Hawks
1,00-0019596,2,Hawks
2,00-0019596,3,Hawks
3,00-0019596,4,Hawks
4,00-0019596,5,Hawks
...,...,...,...
1507,00-0090010,12,Eagles
1508,00-0090010,13,Eagles
1509,00-0090010,14,Eagles
1510,00-0090021,1,Wolves


In [11]:
# Displaying weekly fantasy points scored by different managers
weekly_points = pd.merge(weekly_lineups_merged_adp, fantasy_data_player_rf[['player_id', 'week', 'fantasy_points_ppr']], 
                      left_on=['Player ID', 'Week'], 
                      right_on=['player_id', 'week'], 
                      how='left')

weekly_points = weekly_points.groupby(['Team', 'Week']).agg({'fantasy_points_ppr': 'sum'}).reset_index()

weekly_points = weekly_points.rename(columns={'fantasy_points_ppr': 'Total Points'})

# Displaying Bears game results with Result column
games_team = pd.concat({k: pd.Series(v) for k, v in team_schedule.items()}).reset_index()
games_team.columns = ['Team', 'Week', 'Opponent']

match_score = pd.merge(weekly_points, games_team, how='left', on=['Team', 'Week'])

match_score['Opponent Points'] = match_score.apply(
    lambda row: weekly_points[
        (weekly_points['Team'] == row['Opponent']) &
        (weekly_points['Week'] == row['Week'])
    ]['Total Points'].values[0]
    if row['Opponent'] in weekly_points['Team'].values else 0,
    axis=1
)

match_score['Result'] = np.where(match_score['Total Points'] > match_score['Opponent Points'], 'Win', 'Loss')

team_results = match_score[match_score['Team'] == 'Bears']

print(team_results[['Week', 'Team', 'Total Points', 'Opponent', 'Opponent Points', 'Result']])

    Week   Team  Total Points  Opponent  Opponent Points Result
0      1  Bears         86.34    Eagles             2.10    Win
1      2  Bears         96.70     Hawks            94.74    Win
2      3  Bears        108.22    Sharks            63.70    Win
3      4  Bears         73.44   Falcons            32.70    Win
4      5  Bears         52.20  Panthers            34.20    Win
5      6  Bears         96.42   Cougars            14.20    Win
6      7  Bears         28.70  Dolphins             4.10    Win
7      8  Bears         65.06    Wolves            60.70    Win
8      9  Bears         85.78    Tigers            26.40    Win
9     10  Bears         48.20      Owls            33.40    Win
10    11  Bears        113.84     Lions            37.70    Win
11    12  Bears         53.70    Eagles            11.50    Win
12    13  Bears         48.10     Hawks            43.30    Win
13    14  Bears         55.30    Sharks            38.80    Win


In [12]:
# List of players, teams, and weeks
players = fantasy_data_player_rf['player_id'].unique().tolist()
teams = season_schedule['Team'].unique().tolist()
weeks = list(range(1, 15))
position = dict(zip(fantasy_data_player_rf['player_id'], fantasy_data_player_rf['position_group']))

# Assuming each team selects 16 players
num_players_per_team = 16
rounds = list(range(1, num_players_per_team + 1))

# Create a dictionary to store the rank for each player over the entire season
season_ranking = {row['player_id']: row['rank'] for index, row in consensus_ranks.iterrows()}

# Initialize the model
m = Model("Fantasy_Football_Optimization")

# l[p][k][t] is 1, if player p is selected and played in week k by team t
l = m.addVars(players, weeks, teams, vtype=GRB.BINARY, name="chosen")

# s[p][t] is 1, if player p is selected by team t for the entire season
s = m.addVars(players, teams, vtype=GRB.BINARY, name="selected_by")

# d[t][r] is 1, if team t selects a player in round r
d = m.addVars(teams, rounds, vtype=GRB.BINARY, name="drafted")

# w[t][k] is 1, if team t wins in week k
w = m.addVars(teams, weeks, vtype=GRB.BINARY, name="wins")

# Objective function: Maximize the number of wins for Manager_1
m.setObjective(quicksum(w['Bears', k] for k in weeks), GRB.MAXIMIZE)

# Schedule constraints
for k in weeks:
    opponent = team_schedule['Bears'][k]
    m.addConstr(quicksum(l[p, k, 'Bears'] * projected_points_dict[(p, k)] for p in players) -
                quicksum(l[p, k, opponent] * projected_points_dict[(p, k)] for p in players) >=
                w['Bears', k] - (1 - w[opponent, k]))

for t in teams:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for p in players) == 9)

# Add position constraints
for t in teams:
    for k in weeks:
        # Quarterback Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'QB') == 1)

        # Flex Constraint
        m.addConstr(
            quicksum(l[p, k, t] for p in players if position[p] in ['WR', 'RB']) +
            quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 3
        )

        # Running Back Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'RB') >= 2)

        # Wide Receiver Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'WR') >= 2)

        # Tight End Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 1)

        # Kicker Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'K') == 1)

        # Defense/Special Teams Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'DST') == 1)

for p in players:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for t in teams) <= 1)

for p in players:
    for t in teams:
        m.addConstr(s[p, t] >= quicksum(l[p, k, t] for k in weeks) / len(weeks))

for p in players:
    m.addConstr(quicksum(s[p, t] for t in teams) <= 1)

for t in teams:
    if t != 'Bears':
        m.addConstr(quicksum(s[p, t] * season_ranking[p] for p in players if p in season_ranking and (p, t) in s) <= len(players))

for t in teams:
    m.addConstr(quicksum(s[p, t] for p in players) <= num_players_per_team)

for r in rounds:
    for t in draft_order[r-1]:
        m.addConstr(d[t, r] == 1)

# Solve the problem
m.optimize()

# Output the results
for v in m.getVars():
    if v.x > 0:
        print('%s : %g' % (v.varName, v.x))

# Extract selected players for each team and week
selected_players_consensus = [(p, k, t) for p in players for k in weeks for t in teams if l[p, k, t].x == 1]

# Convert the list to a DataFrame for better presentation
weekly_lineups_consensus = pd.DataFrame(selected_players_consensus, columns=['Player ID', 'Week', 'Team'])

weekly_lineups_consensus


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 19658 rows, 121140 columns and 579719 nonzeros
Model fingerprint: 0x6c9d764a
Variable types: 0 continuous, 121140 integer (121140 binary)
Coefficient statistics:
  Matrix range     [7e-02, 5e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]
Presolve removed 328 rows and 346 columns
Presolve time: 1.98s
Presolved: 19330 rows, 120794 columns, 466797 nonzeros
Variable types: 0 continuous, 120794 integer (120794 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    8505   -0.0000000e+00   2.866281e+00   9.041387e+09      5s
    8695   -0.0000000e+00   0.000000e+00 

,Player ID,Week,Team
0,00-0019596,1,Bears
1,00-0019596,2,Bears
2,00-0019596,3,Bears
3,00-0019596,5,Bears
4,00-0019596,7,Bears
...,...,...,...
1507,00-0090021,10,Owls
1508,00-0090021,11,Owls
1509,00-0090021,12,Owls
1510,00-0090021,13,Owls


In [13]:
# Displaying weekly fantasy points scored by different managers
weekly_points = pd.merge(weekly_lineups_consensus, fantasy_data_player_rf[['player_id', 'week', 'fantasy_points_ppr']], 
                      left_on=['Player ID', 'Week'], 
                      right_on=['player_id', 'week'], 
                      how='left')

weekly_points = weekly_points.groupby(['Team', 'Week']).agg({'fantasy_points_ppr': 'sum'}).reset_index()

weekly_points = weekly_points.rename(columns={'fantasy_points_ppr': 'Total Points'})

# Displaying Bears game results with Result column
games_team = pd.concat({k: pd.Series(v) for k, v in team_schedule.items()}).reset_index()
games_team.columns = ['Team', 'Week', 'Opponent']

match_score = pd.merge(weekly_points, games_team, how='left', on=['Team', 'Week'])

match_score['Opponent Points'] = match_score.apply(
    lambda row: weekly_points[
        (weekly_points['Team'] == row['Opponent']) &
        (weekly_points['Week'] == row['Week'])
    ]['Total Points'].values[0]
    if row['Opponent'] in weekly_points['Team'].values else 0,
    axis=1
)

match_score['Result'] = np.where(match_score['Total Points'] > match_score['Opponent Points'], 'Win', 'Loss')

team_results = match_score[match_score['Team'] == 'Bears']

print(team_results[['Week', 'Team', 'Total Points', 'Opponent', 'Opponent Points', 'Result']])

    Week   Team  Total Points  Opponent  Opponent Points Result
0      1  Bears         71.76    Eagles            17.50    Win
1      2  Bears         74.84     Hawks            45.30    Win
2      3  Bears         77.58    Sharks            71.80    Win
3      4  Bears         37.78   Falcons            53.30   Loss
4      5  Bears        110.94  Panthers            47.24    Win
5      6  Bears        102.24   Cougars            24.50    Win
6      7  Bears         40.64  Dolphins            28.10    Win
7      8  Bears         76.80    Wolves            38.50    Win
8      9  Bears         45.86    Tigers            32.40    Win
9     10  Bears         42.40      Owls            51.90   Loss
10    11  Bears         51.88     Lions            35.60    Win
11    12  Bears         22.30    Eagles            18.80    Win
12    13  Bears         47.70     Hawks            37.20    Win
13    14  Bears         59.88    Sharks            30.90    Win


In [14]:
# List of players, teams, and weeks
players = fantasy_data_player_rf['player_id'].unique().tolist()
teams = season_schedule['Team'].unique().tolist()
weeks = list(range(1, 15))
position = dict(zip(fantasy_data_player_rf['player_id'], fantasy_data_player_rf['position_group']))

# Assuming each team selects 16 players
num_players_per_team = 16
rounds = list(range(1, num_players_per_team + 1))

# Create a dictionary to store the rank for each player over the entire season
season_ranking = {row['player_id']: row['rank'] for index, row in opportunity_cost_avg_ranks.iterrows()}

# Initialize the model
m = Model("Fantasy_Football_Optimization")

# l[p][k][t] is 1, if player p is selected and played in week k by team t
l = m.addVars(players, weeks, teams, vtype=GRB.BINARY, name="chosen")

# s[p][t] is 1, if player p is selected by team t for the entire season
s = m.addVars(players, teams, vtype=GRB.BINARY, name="selected_by")

# d[t][r] is 1, if team t selects a player in round r
d = m.addVars(teams, rounds, vtype=GRB.BINARY, name="drafted")

# w[t][k] is 1, if team t wins in week k
w = m.addVars(teams, weeks, vtype=GRB.BINARY, name="wins")

# Objective function: Maximize the number of wins for Manager_1
m.setObjective(quicksum(w['Bears', k] for k in weeks), GRB.MAXIMIZE)

# Schedule constraints
for k in weeks:
    opponent = team_schedule['Bears'][k]
    m.addConstr(quicksum(l[p, k, 'Bears'] * projected_points_dict[(p, k)] for p in players) -
                quicksum(l[p, k, opponent] * projected_points_dict[(p, k)] for p in players) >=
                w['Bears', k] - (1 - w[opponent, k]))

for t in teams:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for p in players) == 9)

# Add position constraints
for t in teams:
    for k in weeks:
        # Quarterback Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'QB') == 1)

        # Flex Constraint
        m.addConstr(
            quicksum(l[p, k, t] for p in players if position[p] in ['WR', 'RB']) +
            quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 3
        )

        # Running Back Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'RB') >= 2)

        # Wide Receiver Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'WR') >= 2)

        # Tight End Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 1)

        # Kicker Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'K') == 1)

        # Defense/Special Teams Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'DST') == 1)

for p in players:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for t in teams) <= 1)

for p in players:
    for t in teams:
        m.addConstr(s[p, t] >= quicksum(l[p, k, t] for k in weeks) / len(weeks))

for p in players:
    m.addConstr(quicksum(s[p, t] for t in teams) <= 1)

for t in teams:
    if t != 'Bears':
        m.addConstr(quicksum(s[p, t] * season_ranking[p] for p in players if p in season_ranking and (p, t) in s) <= len(players))

for t in teams:
    m.addConstr(quicksum(s[p, t] for p in players) <= num_players_per_team)

for r in rounds:
    for t in draft_order[r-1]:
        m.addConstr(d[t, r] == 1)

# Solve the problem
m.optimize()

# Output the results
for v in m.getVars():
    if v.x > 0:
        print('%s : %g' % (v.varName, v.x))

# Extract selected players for each team and week
selected_players_opp_avg = [(p, k, t) for p in players for k in weeks for t in teams if l[p, k, t].x == 1]

# Convert the list to a DataFrame for better presentation
weekly_lineups_opp_avg = pd.DataFrame(selected_players_opp_avg, columns=['Player ID', 'Week', 'Team'])

weekly_lineups_opp_avg


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 19658 rows, 121140 columns and 577838 nonzeros
Model fingerprint: 0xf57de67f
Variable types: 0 continuous, 121140 integer (121140 binary)
Coefficient statistics:
  Matrix range     [7e-02, 7e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]
Presolve removed 328 rows and 346 columns
Presolve time: 1.95s
Presolved: 19330 rows, 120794 columns, 464916 nonzeros
Variable types: 0 continuous, 120794 integer (120794 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.03s

Solved with dual simplex

Root relaxation: objective 1.400000e+01, 11170 iterations, 1.56 seconds (0.46 work units)
Total elapsed time = 6.08s

    Nodes    |    

,Player ID,Week,Team
0,00-0023459,5,Panthers
1,00-0023459,6,Panthers
2,00-0023459,7,Panthers
3,00-0023459,8,Panthers
4,00-0023459,12,Panthers
...,...,...,...
1507,00-0090010,10,Lions
1508,00-0090010,11,Lions
1509,00-0090010,12,Lions
1510,00-0090010,13,Lions


In [15]:
# Displaying weekly fantasy points scored by different managers
weekly_points = pd.merge(weekly_lineups_opp_avg, fantasy_data_player_rf[['player_id', 'week', 'fantasy_points_ppr']], 
                      left_on=['Player ID', 'Week'], 
                      right_on=['player_id', 'week'], 
                      how='left')

weekly_points = weekly_points.groupby(['Team', 'Week']).agg({'fantasy_points_ppr': 'sum'}).reset_index()

weekly_points = weekly_points.rename(columns={'fantasy_points_ppr': 'Total Points'})

# Displaying Bears game results with Result column
games_team = pd.concat({k: pd.Series(v) for k, v in team_schedule.items()}).reset_index()
games_team.columns = ['Team', 'Week', 'Opponent']

match_score = pd.merge(weekly_points, games_team, how='left', on=['Team', 'Week'])

match_score['Opponent Points'] = match_score.apply(
    lambda row: weekly_points[
        (weekly_points['Team'] == row['Opponent']) &
        (weekly_points['Week'] == row['Week'])
    ]['Total Points'].values[0]
    if row['Opponent'] in weekly_points['Team'].values else 0,
    axis=1
)

match_score['Result'] = np.where(match_score['Total Points'] > match_score['Opponent Points'], 'Win', 'Loss')

team_results = match_score[match_score['Team'] == 'Bears']

print(team_results[['Week', 'Team', 'Total Points', 'Opponent', 'Opponent Points', 'Result']])

    Week   Team  Total Points  Opponent  Opponent Points Result
0      1  Bears         95.82    Eagles            26.40    Win
1      2  Bears         59.64     Hawks            28.50    Win
2      3  Bears         71.28    Sharks            46.72    Win
3      4  Bears         52.86   Falcons            12.40    Win
4      5  Bears         79.72  Panthers            29.16    Win
5      6  Bears         69.48   Cougars            28.00    Win
6      7  Bears         41.62  Dolphins            24.00    Win
7      8  Bears         43.78    Wolves            23.80    Win
8      9  Bears         55.70    Tigers            42.40    Win
9     10  Bears         38.66      Owls            42.30   Loss
10    11  Bears         30.80     Lions            17.10    Win
11    12  Bears         45.34    Eagles             7.00    Win
12    13  Bears         78.94     Hawks            16.30    Win
13    14  Bears         41.20    Sharks            38.30    Win


In [16]:
# List of players, teams, and weeks
players = fantasy_data_player_rf['player_id'].unique().tolist()
teams = season_schedule['Team'].unique().tolist()
weeks = list(range(1, 15))
position = dict(zip(fantasy_data_player_rf['player_id'], fantasy_data_player_rf['position_group']))

# Assuming each team selects 16 players
num_players_per_team = 16
rounds = list(range(1, num_players_per_team + 1))

# Create a dictionary to store the rank for each player over the entire season
season_ranking = {row['player_id']: row['rank'] for index, row in opportunity_cost_ranks.iterrows()}

# Initialize the model
m = Model("Fantasy_Football_Optimization")

# p[i][k][t] is 1, if player i is selected and played in week k by team t
l = m.addVars(players, weeks, teams, vtype=GRB.BINARY, name="chosen")

# s[i][t] is 1, if player i is selected by team t for the entire season
s = m.addVars(players, teams, vtype=GRB.BINARY, name="selected_by")

# d[t][r] is 1, if team t selects a player in round r
d = m.addVars(teams, rounds, vtype=GRB.BINARY, name="drafted")

# w[t][k] is 1, if team t wins in week k
w = m.addVars(teams, weeks, vtype=GRB.BINARY, name="wins")

# Objective function: Maximize the number of wins for Manager_1
m.setObjective(quicksum(w['Bears', k] for k in weeks), GRB.MAXIMIZE)

# Schedule constraints
for k in weeks:
    opponent = team_schedule['Bears'][k]
    m.addConstr(quicksum(l[p, k, 'Bears'] * projected_points_dict[(p, k)] for p in players) -
                quicksum(l[p, k, opponent] * projected_points_dict[(p, k)] for p in players) >=
                w['Bears', k] - (1 - w[opponent, k]))

for t in teams:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for p in players) == 9)

# Add position constraints
for t in teams:
    for k in weeks:
        # Quarterback Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'QB') == 1)

        # Flex Constraint
        m.addConstr(
            quicksum(l[p, k, t] for p in players if position[p] in ['WR', 'RB']) +
            quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 3
        )

        # Running Back Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'RB') >= 2)

        # Wide Receiver Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'WR') >= 2)

        # Tight End Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 1)

        # Kicker Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'K') == 1)

        # Defense/Special Teams Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'DST') == 1)

for p in players:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for t in teams) <= 1)

for p in players:
    for t in teams:
        m.addConstr(s[p, t] >= quicksum(l[p, k, t] for k in weeks) / len(weeks))

for p in players:
    m.addConstr(quicksum(s[p, t] for t in teams) <= 1)

for t in teams:
    if t != 'Bears':
        m.addConstr(quicksum(s[p, t] * season_ranking[p] for p in players if p in season_ranking and (p, t) in s) <= len(players))

for t in teams:
    m.addConstr(quicksum(s[p, t] for p in players) <= num_players_per_team)

for r in rounds:
    for t in draft_order[r-1]:
        m.addConstr(d[t, r] == 1)

# Solve the problem
m.optimize()

# Output the results
for v in m.getVars():
    if v.x > 0:
        print('%s : %g' % (v.varName, v.x))

# Extract selected players for each team and week
selected_players_opp = [(p, k, t) for p in players for k in weeks for t in teams if l[p, k, t].x == 1]

# Convert the list to a DataFrame for better presentation
weekly_lineups_opp = pd.DataFrame(selected_players_opp, columns=['Player ID', 'Week', 'Team'])

weekly_lineups_opp


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 19658 rows, 121140 columns and 577838 nonzeros
Model fingerprint: 0x3d39f2dd
Variable types: 0 continuous, 121140 integer (121140 binary)
Coefficient statistics:
  Matrix range     [7e-02, 4e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]
Presolve removed 328 rows and 346 columns
Presolve time: 1.84s
Presolved: 19330 rows, 120794 columns, 464916 nonzeros
Variable types: 0 continuous, 120794 integer (120794 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.02s

Solved with dual simplex

Root relaxation: objective 1.400000e+01, 11136 iterations, 1.70 seconds (0.46 work units)
Total elapsed time = 6.11s

    Nodes    |    

,Player ID,Week,Team
0,00-0023459,1,Panthers
1,00-0023459,2,Panthers
2,00-0023459,3,Panthers
3,00-0023459,4,Panthers
4,00-0023459,5,Panthers
...,...,...,...
1507,00-0090010,10,Falcons
1508,00-0090010,11,Falcons
1509,00-0090010,12,Falcons
1510,00-0090010,13,Falcons


In [17]:
# Displaying weekly fantasy points scored by different managers
weekly_points = pd.merge(weekly_lineups_opp, fantasy_data_player_rf[['player_id', 'week', 'fantasy_points_ppr']], 
                      left_on=['Player ID', 'Week'], 
                      right_on=['player_id', 'week'], 
                      how='left')

weekly_points = weekly_points.groupby(['Team', 'Week']).agg({'fantasy_points_ppr': 'sum'}).reset_index()

weekly_points = weekly_points.rename(columns={'fantasy_points_ppr': 'Total Points'})

# Displaying Bears game results with Result column
games_team = pd.concat({k: pd.Series(v) for k, v in team_schedule.items()}).reset_index()
games_team.columns = ['Team', 'Week', 'Opponent']

match_score = pd.merge(weekly_points, games_team, how='left', on=['Team', 'Week'])

match_score['Opponent Points'] = match_score.apply(
    lambda row: weekly_points[
        (weekly_points['Team'] == row['Opponent']) &
        (weekly_points['Week'] == row['Week'])
    ]['Total Points'].values[0]
    if row['Opponent'] in weekly_points['Team'].values else 0,
    axis=1
)

match_score['Result'] = np.where(match_score['Total Points'] > match_score['Opponent Points'], 'Win', 'Loss')

team_results = match_score[match_score['Team'] == 'Bears']

print(team_results[['Week', 'Team', 'Total Points', 'Opponent', 'Opponent Points', 'Result']])

    Week   Team  Total Points  Opponent  Opponent Points Result
0      1  Bears         35.50    Eagles            38.46   Loss
1      2  Bears         37.26     Hawks            31.20    Win
2      3  Bears         64.92    Sharks            22.62    Win
3      4  Bears         57.62   Falcons            37.50    Win
4      5  Bears         80.40  Panthers            59.66    Win
5      6  Bears         64.32   Cougars            51.20    Win
6      7  Bears         24.10  Dolphins            13.90    Win
7      8  Bears         51.26    Wolves            45.50    Win
8      9  Bears         29.96    Tigers            13.00    Win
9     10  Bears         38.04      Owls            27.24    Win
10    11  Bears         53.06     Lions            34.00    Win
11    12  Bears         63.10    Eagles            18.96    Win
12    13  Bears         49.10     Hawks            35.30    Win
13    14  Bears         73.82    Sharks            32.90    Win


In [18]:
# Set Multi-index for Fantasy Player Data
fantasy_data_player_xgb.set_index(['player_id', 'week'], inplace=True)

# Generate Matrix of Expected Fantasy Points
projected_points_xgb = fantasy_data_player_xgb.pivot_table(values='predicted_fantasy_points', index=['player_id'], columns=['week'], fill_value=0)

# Reset Index for Fantasy Player Data
fantasy_data_player_xgb = fantasy_data_player_xgb.reset_index()

# Transform to Long-format DataFrame
full_projected_points_xgb = projected_points_xgb.reset_index().melt(id_vars='player_id', var_name='week', value_name='points')
full_projected_points_xgb['week'] = full_projected_points_xgb['week'].astype(int)  # Ensure 'week' is an integer

# Create a dictionary from the long-format DataFrame
projected_points_dict_xgb = {(row['player_id'], row['week']): row['points'] for index, row in full_projected_points_xgb.iterrows()}

# # Display the dictionary or DataFrame
# print(projected_points_dict_xgb)


In [19]:
# List of players, teams, and weeks
players = fantasy_data_player_xgb['player_id'].unique().tolist()
teams = season_schedule['Team'].unique().tolist()
weeks = list(range(1, 15))
position = dict(zip(fantasy_data_player_xgb['player_id'], fantasy_data_player_xgb['position_group']))

# Assuming each team selects 16 players
num_players_per_team = 16
rounds = list(range(1, num_players_per_team + 1))

# Create a dictionary to store the rank for each player over the entire season
season_ranking = {row['player_id']: row['rank'] for index, row in adp_and_vor_ranks.iterrows()}

# Initialize the model
m = Model("Fantasy_Football_Optimization")

# l[p][k][t] is 1, if player p is selected and played in week k by team t
l = m.addVars(players, weeks, teams, vtype=GRB.BINARY, name="played")

# s[p][t] is 1, if player p is selected by team t for the entire season
s = m.addVars(players, teams, vtype=GRB.BINARY, name="selected_by")

# d[t][r] is 1, if team t selects a player in round r
d = m.addVars(teams, rounds, vtype=GRB.BINARY, name="drafted")

# w[t][k] is 1, if team t wins in week k
w = m.addVars(teams, weeks, vtype=GRB.BINARY, name="wins")

# Objective function: Maximize the number of wins for Team Bears
m.setObjective(quicksum(w['Bears', k] for k in weeks), GRB.MAXIMIZE)

# Schedule constraints
for k in weeks:
    opponent = team_schedule['Bears'][k]
    m.addConstr(quicksum(l[p, k, 'Bears'] * projected_points_dict_xgb[(p, k)] for p in players) -
                quicksum(l[p, k, opponent] * projected_points_dict_xgb[(p, k)] for p in players) >=
                w['Bears', k] - (1 - w[opponent, k]))

for t in teams:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for p in players) == 9)

# Add position constraints
for t in teams:
    for k in weeks:
        # Quarterback Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'QB') == 1)

        # Flex Constraint
        m.addConstr(
            quicksum(l[p, k, t] for p in players if position[p] in ['WR', 'RB']) +
            quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 3
        )

        # Running Back Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'RB') >= 2)

        # Wide Receiver Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'WR') >= 2)

        # Tight End Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 1)

        # Kicker Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'K') == 1)

        # Defense/Special Teams Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'DST') == 1)

for p in players:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for t in teams) <= 1)

for p in players:
    for t in teams:
        m.addConstr(s[p, t] >= quicksum(l[p, k, t] for k in weeks) / len(weeks))

for p in players:
    m.addConstr(quicksum(s[p, t] for t in teams) <= 1)

for t in teams:
    if t != 'Bears':
        m.addConstr(quicksum(s[p, t] * season_ranking[p] for p in players if p in season_ranking and (p, t) in s) <= len(players))

for t in teams:
    m.addConstr(quicksum(s[p, t] for p in players) <= num_players_per_team)

for r in rounds:
    for t in draft_order[r-1]:
        m.addConstr(d[t, r] == 1)

# Solve the problem
m.optimize()

# Output the results
for v in m.getVars():
    if v.x > 0:
        print('%s : %g' % (v.varName, v.x))

# Extract selected players for each team and week
selected_players_adp_vor = [(p, k, t) for p in players for k in weeks for t in teams if l[p, k, t].x == 1]

# Convert the list to a DataFrame for better presentation
weekly_lineups_adp_vor = pd.DataFrame(selected_players_adp_vor, columns=['Player ID', 'Week', 'Team'])

weekly_lineups_adp_vor

# Displaying weekly fantasy points scored by different managers
weekly_points = pd.merge(weekly_lineups_adp_vor, fantasy_data_player_xgb[['player_id', 'week', 'fantasy_points_ppr']], 
                      left_on=['Player ID', 'Week'], 
                      right_on=['player_id', 'week'], 
                      how='left')

weekly_points = weekly_points.groupby(['Team', 'Week']).agg({'fantasy_points_ppr': 'sum'}).reset_index()

weekly_points = weekly_points.rename(columns={'fantasy_points_ppr': 'Total Points'})

# Displaying Bears game results with Result column
games_team = pd.concat({k: pd.Series(v) for k, v in team_schedule.items()}).reset_index()
games_team.columns = ['Team', 'Week', 'Opponent']

match_score = pd.merge(weekly_points, games_team, how='left', on=['Team', 'Week'])

match_score['Opponent Points'] = match_score.apply(
    lambda row: weekly_points[
        (weekly_points['Team'] == row['Opponent']) &
        (weekly_points['Week'] == row['Week'])
    ]['Total Points'].values[0]
    if row['Opponent'] in weekly_points['Team'].values else 0,
    axis=1
)

match_score['Result'] = np.where(match_score['Total Points'] > match_score['Opponent Points'], 'Win', 'Loss')

team_results = match_score[match_score['Team'] == 'Bears']

print(team_results[['Week', 'Team', 'Total Points', 'Opponent', 'Opponent Points', 'Result']])


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 19658 rows, 121140 columns and 577838 nonzeros
Model fingerprint: 0x0f18d037
Variable types: 0 continuous, 121140 integer (121140 binary)
Coefficient statistics:
  Matrix range     [4e-03, 4e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]
Presolve removed 328 rows and 346 columns
Presolve time: 2.46s
Presolved: 19330 rows, 120794 columns, 464916 nonzeros
Variable types: 0 continuous, 120794 integer (120794 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -0.0000000e+00   8.465427e+02   1.497601e+10      5s
Concurrent spin time: 0.00s

Solved with 

In [20]:
# List of players, teams, and weeks
players = fantasy_data_player_xgb['player_id'].unique().tolist()
teams = season_schedule['Team'].unique().tolist()
weeks = list(range(1, 15))
position = dict(zip(fantasy_data_player_xgb['player_id'], fantasy_data_player_xgb['position_group']))

# Assuming each team selects 16 players
num_players_per_team = 16
rounds = list(range(1, num_players_per_team + 1))

# Create a dictionary to store the rank for each player over the entire season
season_ranking = {row['player_id']: row['rank'] for index, row in merged_adp_ranks.iterrows()}

# Initialize the model
m = Model("Fantasy_Football_Optimization")

# l[p][k][t] is 1, if player p is selected and played in week k by team t
l = m.addVars(players, weeks, teams, vtype=GRB.BINARY, name="played")

# s[p][t] is 1, if player p is selected by team t for the entire season
s = m.addVars(players, teams, vtype=GRB.BINARY, name="selected_by")

# d[t][r] is 1, if team t selects a player in round r
d = m.addVars(teams, rounds, vtype=GRB.BINARY, name="drafted")

# w[t][k] is 1, if team t wins in week k
w = m.addVars(teams, weeks, vtype=GRB.BINARY, name="wins")

# Objective function: Maximize the number of wins for Team Bears
m.setObjective(quicksum(w['Bears', k] for k in weeks), GRB.MAXIMIZE)

# Schedule constraints
for k in weeks:
    opponent = team_schedule['Bears'][k]
    m.addConstr(quicksum(l[p, k, 'Bears'] * projected_points_dict_xgb[(p, k)] for p in players) -
                quicksum(l[p, k, opponent] * projected_points_dict_xgb[(p, k)] for p in players) >=
                w['Bears', k] - (1 - w[opponent, k]))

for t in teams:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for p in players) == 9)

# Add position constraints
for t in teams:
    for k in weeks:
        # Quarterback Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'QB') == 1)

        # Flex Constraint
        m.addConstr(
            quicksum(l[p, k, t] for p in players if position[p] in ['WR', 'RB']) +
            quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 3
        )

        # Running Back Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'RB') >= 2)

        # Wide Receiver Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'WR') >= 2)

        # Tight End Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 1)

        # Kicker Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'K') == 1)

        # Defense/Special Teams Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'DST') == 1)

for p in players:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for t in teams) <= 1)

for p in players:
    for t in teams:
        m.addConstr(s[p, t] >= quicksum(l[p, k, t] for k in weeks) / len(weeks))

for p in players:
    m.addConstr(quicksum(s[p, t] for t in teams) <= 1)

for t in teams:
    if t != 'Bears':
        m.addConstr(quicksum(s[p, t] * season_ranking[p] for p in players if p in season_ranking and (p, t) in s) <= len(players))

for t in teams:
    m.addConstr(quicksum(s[p, t] for p in players) <= num_players_per_team)

for r in rounds:
    for t in draft_order[r-1]:
        m.addConstr(d[t, r] == 1)

# Solve the problem
m.optimize()

# Output the results
for v in m.getVars():
    if v.x > 0:
        print('%s : %g' % (v.varName, v.x))

# Extract selected players for each team and week
selected_players_merged_adp = [(p, k, t) for p in players for k in weeks for t in teams if l[p, k, t].x == 1]

# Convert the list to a DataFrame for better presentation
weekly_lineups_merged_adp = pd.DataFrame(selected_players_merged_adp, columns=['Player ID', 'Week', 'Team'])

weekly_lineups_merged_adp

# Displaying weekly fantasy points scored by different managers
weekly_points = pd.merge(weekly_lineups_merged_adp, fantasy_data_player_xgb[['player_id', 'week', 'fantasy_points_ppr']], 
                      left_on=['Player ID', 'Week'], 
                      right_on=['player_id', 'week'], 
                      how='left')

weekly_points = weekly_points.groupby(['Team', 'Week']).agg({'fantasy_points_ppr': 'sum'}).reset_index()

weekly_points = weekly_points.rename(columns={'fantasy_points_ppr': 'Total Points'})

# Displaying Bears game results with Result column
games_team = pd.concat({k: pd.Series(v) for k, v in team_schedule.items()}).reset_index()
games_team.columns = ['Team', 'Week', 'Opponent']

match_score = pd.merge(weekly_points, games_team, how='left', on=['Team', 'Week'])

match_score['Opponent Points'] = match_score.apply(
    lambda row: weekly_points[
        (weekly_points['Team'] == row['Opponent']) &
        (weekly_points['Week'] == row['Week'])
    ]['Total Points'].values[0]
    if row['Opponent'] in weekly_points['Team'].values else 0,
    axis=1
)

match_score['Result'] = np.where(match_score['Total Points'] > match_score['Opponent Points'], 'Win', 'Loss')

team_results = match_score[match_score['Team'] == 'Bears']

print(team_results[['Week', 'Team', 'Total Points', 'Opponent', 'Opponent Points', 'Result']])

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 19658 rows, 121140 columns and 577838 nonzeros
Model fingerprint: 0xfc00db29
Variable types: 0 continuous, 121140 integer (121140 binary)
Coefficient statistics:
  Matrix range     [4e-03, 4e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]
Presolve removed 328 rows and 346 columns
Presolve time: 2.45s
Presolved: 19330 rows, 120794 columns, 464916 nonzeros
Variable types: 0 continuous, 120794 integer (120794 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -0.0000000e+00   9.704079e+02   1.531724e+10      5s
Concurrent spin time: 0.00s

Solved with 

In [21]:
# List of players, teams, and weeks
players = fantasy_data_player_xgb['player_id'].unique().tolist()
teams = season_schedule['Team'].unique().tolist()
weeks = list(range(1, 15))
position = dict(zip(fantasy_data_player_xgb['player_id'], fantasy_data_player_xgb['position_group']))

# Assuming each team selects 16 players
num_players_per_team = 16
rounds = list(range(1, num_players_per_team + 1))

# Create a dictionary to store the rank for each player over the entire season
season_ranking = {row['player_id']: row['rank'] for index, row in consensus_ranks.iterrows()}

# Initialize the model
m = Model("Fantasy_Football_Optimization")

# l[p][k][t] is 1, if player p is selected and played in week k by team t
l = m.addVars(players, weeks, teams, vtype=GRB.BINARY, name="played")

# s[p][t] is 1, if player p is selected by team t for the entire season
s = m.addVars(players, teams, vtype=GRB.BINARY, name="selected_by")

# d[t][r] is 1, if team t selects a player in round r
d = m.addVars(teams, rounds, vtype=GRB.BINARY, name="drafted")

# w[t][k] is 1, if team t wins in week k
w = m.addVars(teams, weeks, vtype=GRB.BINARY, name="wins")

# Objective function: Maximize the number of wins for Team Bears
m.setObjective(quicksum(w['Bears', k] for k in weeks), GRB.MAXIMIZE)

# Schedule constraints
for k in weeks:
    opponent = team_schedule['Bears'][k]
    m.addConstr(quicksum(l[p, k, 'Bears'] * projected_points_dict_xgb[(p, k)] for p in players) -
                quicksum(l[p, k, opponent] * projected_points_dict_xgb[(p, k)] for p in players) >=
                w['Bears', k] - (1 - w[opponent, k]))

for t in teams:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for p in players) == 9)

# Add position constraints
for t in teams:
    for k in weeks:
        # Quarterback Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'QB') == 1)

        # Flex Constraint
        m.addConstr(
            quicksum(l[p, k, t] for p in players if position[p] in ['WR', 'RB']) +
            quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 3
        )

        # Running Back Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'RB') >= 2)

        # Wide Receiver Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'WR') >= 2)

        # Tight End Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 1)

        # Kicker Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'K') == 1)

        # Defense/Special Teams Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'DST') == 1)

for p in players:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for t in teams) <= 1)

for p in players:
    for t in teams:
        m.addConstr(s[p, t] >= quicksum(l[p, k, t] for k in weeks) / len(weeks))

for p in players:
    m.addConstr(quicksum(s[p, t] for t in teams) <= 1)

for t in teams:
    if t != 'Bears':
        m.addConstr(quicksum(s[p, t] * season_ranking[p] for p in players if p in season_ranking and (p, t) in s) <= len(players))

for t in teams:
    m.addConstr(quicksum(s[p, t] for p in players) <= num_players_per_team)

for r in rounds:
    for t in draft_order[r-1]:
        m.addConstr(d[t, r] == 1)

# Solve the problem
m.optimize()

# Output the results
for v in m.getVars():
    if v.x > 0:
        print('%s : %g' % (v.varName, v.x))

# Extract selected players for each team and week
selected_players_consensus = [(p, k, t) for p in players for k in weeks for t in teams if l[p, k, t].x == 1]

# Convert the list to a DataFrame for better presentation
weekly_lineups_consensus = pd.DataFrame(selected_players_consensus, columns=['Player ID', 'Week', 'Team'])

weekly_lineups_consensus

# Displaying weekly fantasy points scored by different managers
weekly_points = pd.merge(weekly_lineups_consensus, fantasy_data_player_xgb[['player_id', 'week', 'fantasy_points_ppr']], 
                      left_on=['Player ID', 'Week'], 
                      right_on=['player_id', 'week'], 
                      how='left')

weekly_points = weekly_points.groupby(['Team', 'Week']).agg({'fantasy_points_ppr': 'sum'}).reset_index()

weekly_points = weekly_points.rename(columns={'fantasy_points_ppr': 'Total Points'})

# Displaying Bears game results with Result column
games_team = pd.concat({k: pd.Series(v) for k, v in team_schedule.items()}).reset_index()
games_team.columns = ['Team', 'Week', 'Opponent']

match_score = pd.merge(weekly_points, games_team, how='left', on=['Team', 'Week'])

match_score['Opponent Points'] = match_score.apply(
    lambda row: weekly_points[
        (weekly_points['Team'] == row['Opponent']) &
        (weekly_points['Week'] == row['Week'])
    ]['Total Points'].values[0]
    if row['Opponent'] in weekly_points['Team'].values else 0,
    axis=1
)

match_score['Result'] = np.where(match_score['Total Points'] > match_score['Opponent Points'], 'Win', 'Loss')

team_results = match_score[match_score['Team'] == 'Bears']

print(team_results[['Week', 'Team', 'Total Points', 'Opponent', 'Opponent Points', 'Result']])

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 19658 rows, 121140 columns and 579719 nonzeros
Model fingerprint: 0xea18d9fe
Variable types: 0 continuous, 121140 integer (121140 binary)
Coefficient statistics:
  Matrix range     [4e-03, 5e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]
Presolve removed 328 rows and 346 columns
Presolve time: 2.23s
Presolved: 19330 rows, 120794 columns, 466797 nonzeros
Variable types: 0 continuous, 120794 integer (120794 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    5559   -0.0000000e+00   1.150365e+02   3.828860e+10      5s
    9033   -0.0000000e+00   0.000000e+00 

In [22]:
# List of players, teams, and weeks
players = fantasy_data_player_xgb['player_id'].unique().tolist()
teams = season_schedule['Team'].unique().tolist()
weeks = list(range(1, 15))
position = dict(zip(fantasy_data_player_xgb['player_id'], fantasy_data_player_xgb['position_group']))

# Assuming each team selects 16 players
num_players_per_team = 16
rounds = list(range(1, num_players_per_team + 1))

# Create a dictionary to store the rank for each player over the entire season
season_ranking = {row['player_id']: row['rank'] for index, row in opportunity_cost_avg_ranks.iterrows()}

# Initialize the model
m = Model("Fantasy_Football_Optimization")

# l[p][k][t] is 1, if player p is selected and played in week k by team t
l = m.addVars(players, weeks, teams, vtype=GRB.BINARY, name="played")

# s[p][t] is 1, if player p is selected by team t for the entire season
s = m.addVars(players, teams, vtype=GRB.BINARY, name="selected_by")

# d[t][r] is 1, if team t selects a player in round r
d = m.addVars(teams, rounds, vtype=GRB.BINARY, name="drafted")

# w[t][k] is 1, if team t wins in week k
w = m.addVars(teams, weeks, vtype=GRB.BINARY, name="wins")

# Objective function: Maximize the number of wins for Team Bears
m.setObjective(quicksum(w['Bears', k] for k in weeks), GRB.MAXIMIZE)

# Schedule constraints
for k in weeks:
    opponent = team_schedule['Bears'][k]
    m.addConstr(quicksum(l[p, k, 'Bears'] * projected_points_dict_xgb[(p, k)] for p in players) -
                quicksum(l[p, k, opponent] * projected_points_dict_xgb[(p, k)] for p in players) >=
                w['Bears', k] - (1 - w[opponent, k]))

for t in teams:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for p in players) == 9)

# Add position constraints
for t in teams:
    for k in weeks:
        # Quarterback Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'QB') == 1)

        # Flex Constraint
        m.addConstr(
            quicksum(l[p, k, t] for p in players if position[p] in ['WR', 'RB']) +
            quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 3
        )

        # Running Back Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'RB') >= 2)

        # Wide Receiver Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'WR') >= 2)

        # Tight End Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 1)

        # Kicker Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'K') == 1)

        # Defense/Special Teams Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'DST') == 1)

for p in players:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for t in teams) <= 1)

for p in players:
    for t in teams:
        m.addConstr(s[p, t] >= quicksum(l[p, k, t] for k in weeks) / len(weeks))

for p in players:
    m.addConstr(quicksum(s[p, t] for t in teams) <= 1)

for t in teams:
    if t != 'Bears':
        m.addConstr(quicksum(s[p, t] * season_ranking[p] for p in players if p in season_ranking and (p, t) in s) <= len(players))

for t in teams:
    m.addConstr(quicksum(s[p, t] for p in players) <= num_players_per_team)

for r in rounds:
    for t in draft_order[r-1]:
        m.addConstr(d[t, r] == 1)

# Solve the problem
m.optimize()

# Output the results
for v in m.getVars():
    if v.x > 0:
        print('%s : %g' % (v.varName, v.x))

# Extract selected players for each team and week
selected_players_opp_avg = [(p, k, t) for p in players for k in weeks for t in teams if l[p, k, t].x == 1]

# Convert the list to a DataFrame for better presentation
weekly_lineups_opp_avg = pd.DataFrame(selected_players_opp_avg, columns=['Player ID', 'Week', 'Team'])

weekly_lineups_opp_avg

# Displaying weekly fantasy points scored by different managers
weekly_points = pd.merge(weekly_lineups_opp_avg, fantasy_data_player_xgb[['player_id', 'week', 'fantasy_points_ppr']], 
                      left_on=['Player ID', 'Week'], 
                      right_on=['player_id', 'week'], 
                      how='left')

weekly_points = weekly_points.groupby(['Team', 'Week']).agg({'fantasy_points_ppr': 'sum'}).reset_index()

weekly_points = weekly_points.rename(columns={'fantasy_points_ppr': 'Total Points'})

# Displaying Bears game results with Result column
games_team = pd.concat({k: pd.Series(v) for k, v in team_schedule.items()}).reset_index()
games_team.columns = ['Team', 'Week', 'Opponent']

match_score = pd.merge(weekly_points, games_team, how='left', on=['Team', 'Week'])

match_score['Opponent Points'] = match_score.apply(
    lambda row: weekly_points[
        (weekly_points['Team'] == row['Opponent']) &
        (weekly_points['Week'] == row['Week'])
    ]['Total Points'].values[0]
    if row['Opponent'] in weekly_points['Team'].values else 0,
    axis=1
)

match_score['Result'] = np.where(match_score['Total Points'] > match_score['Opponent Points'], 'Win', 'Loss')

team_results = match_score[match_score['Team'] == 'Bears']

print(team_results[['Week', 'Team', 'Total Points', 'Opponent', 'Opponent Points', 'Result']])

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 19658 rows, 121140 columns and 577838 nonzeros
Model fingerprint: 0x39590e13
Variable types: 0 continuous, 121140 integer (121140 binary)
Coefficient statistics:
  Matrix range     [4e-03, 7e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]
Presolve removed 328 rows and 346 columns
Presolve time: 2.10s
Presolved: 19330 rows, 120794 columns, 464916 nonzeros
Variable types: 0 continuous, 120794 integer (120794 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.01s

Solved with dual simplex

Root relaxation: objective 1.400000e+01, 11048 iterations, 1.47 seconds (0.46 work units)
Total elapsed time = 6.17s

    Nodes    |    

In [23]:
# List of players, teams, and weeks
players = fantasy_data_player_xgb['player_id'].unique().tolist()
teams = season_schedule['Team'].unique().tolist()
weeks = list(range(1, 15))
position = dict(zip(fantasy_data_player_xgb['player_id'], fantasy_data_player_xgb['position_group']))

# Assuming each team selects 16 players
num_players_per_team = 16
rounds = list(range(1, num_players_per_team + 1))

# Create a dictionary to store the rank for each player over the entire season
season_ranking = {row['player_id']: row['rank'] for index, row in opportunity_cost_ranks.iterrows()}

# Initialize the model
m = Model("Fantasy_Football_Optimization")

# l[p][k][t] is 1, if player p is selected and played in week k by team t
l = m.addVars(players, weeks, teams, vtype=GRB.BINARY, name="played")

# s[p][t] is 1, if player p is selected by team t for the entire season
s = m.addVars(players, teams, vtype=GRB.BINARY, name="selected_by")

# d[t][r] is 1, if team t selects a player in round r
d = m.addVars(teams, rounds, vtype=GRB.BINARY, name="drafted")

# w[t][k] is 1, if team t wins in week k
w = m.addVars(teams, weeks, vtype=GRB.BINARY, name="wins")

# Objective function: Maximize the number of wins for Team Bears
m.setObjective(quicksum(w['Bears', k] for k in weeks), GRB.MAXIMIZE)

# Schedule constraints
for k in weeks:
    opponent = team_schedule['Bears'][k]
    m.addConstr(quicksum(l[p, k, 'Bears'] * projected_points_dict_xgb[(p, k)] for p in players) -
                quicksum(l[p, k, opponent] * projected_points_dict_xgb[(p, k)] for p in players) >=
                w['Bears', k] - (1 - w[opponent, k]))

for t in teams:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for p in players) == 9)

# Add position constraints
for t in teams:
    for k in weeks:
        # Quarterback Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'QB') == 1)

        # Flex Constraint
        m.addConstr(
            quicksum(l[p, k, t] for p in players if position[p] in ['WR', 'RB']) +
            quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 3
        )

        # Running Back Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'RB') >= 2)

        # Wide Receiver Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'WR') >= 2)

        # Tight End Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'TE') >= 1)

        # Kicker Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'K') == 1)

        # Defense/Special Teams Constraint
        m.addConstr(quicksum(l[p, k, t] for p in players if position[p] == 'DST') == 1)

for p in players:
    for k in weeks:
        m.addConstr(quicksum(l[p, k, t] for t in teams) <= 1)

for p in players:
    for t in teams:
        m.addConstr(s[p, t] >= quicksum(l[p, k, t] for k in weeks) / len(weeks))

for p in players:
    m.addConstr(quicksum(s[p, t] for t in teams) <= 1)

for t in teams:
    if t != 'Bears':
        m.addConstr(quicksum(s[p, t] * season_ranking[p] for p in players if p in season_ranking and (p, t) in s) <= len(players))

for t in teams:
    m.addConstr(quicksum(s[p, t] for p in players) <= num_players_per_team)

for r in rounds:
    for t in draft_order[r-1]:
        m.addConstr(d[t, r] == 1)

# Solve the problem
m.optimize()

# Output the results
for v in m.getVars():
    if v.x > 0:
        print('%s : %g' % (v.varName, v.x))

# Extract selected players for each team and week
selected_players_opp = [(p, k, t) for p in players for k in weeks for t in teams if l[p, k, t].x == 1]

# Convert the list to a DataFrame for better presentation
weekly_lineups_opp = pd.DataFrame(selected_players_opp, columns=['Player ID', 'Week', 'Team'])

weekly_lineups_opp

# Displaying weekly fantasy points scored by different managers
weekly_points = pd.merge(weekly_lineups_opp, fantasy_data_player_xgb[['player_id', 'week', 'fantasy_points_ppr']], 
                      left_on=['Player ID', 'Week'], 
                      right_on=['player_id', 'week'], 
                      how='left')

weekly_points = weekly_points.groupby(['Team', 'Week']).agg({'fantasy_points_ppr': 'sum'}).reset_index()

weekly_points = weekly_points.rename(columns={'fantasy_points_ppr': 'Total Points'})

# Displaying Bears game results with Result column
games_team = pd.concat({k: pd.Series(v) for k, v in team_schedule.items()}).reset_index()
games_team.columns = ['Team', 'Week', 'Opponent']

match_score = pd.merge(weekly_points, games_team, how='left', on=['Team', 'Week'])

match_score['Opponent Points'] = match_score.apply(
    lambda row: weekly_points[
        (weekly_points['Team'] == row['Opponent']) &
        (weekly_points['Week'] == row['Week'])
    ]['Total Points'].values[0]
    if row['Opponent'] in weekly_points['Team'].values else 0,
    axis=1
)

match_score['Result'] = np.where(match_score['Total Points'] > match_score['Opponent Points'], 'Win', 'Loss')

team_results = match_score[match_score['Team'] == 'Bears']

print(team_results[['Week', 'Team', 'Total Points', 'Opponent', 'Opponent Points', 'Result']])

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 19658 rows, 121140 columns and 577838 nonzeros
Model fingerprint: 0xef4243b7
Variable types: 0 continuous, 121140 integer (121140 binary)
Coefficient statistics:
  Matrix range     [4e-03, 4e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]
Presolve removed 328 rows and 346 columns
Presolve time: 2.03s
Presolved: 19330 rows, 120794 columns, 464916 nonzeros
Variable types: 0 continuous, 120794 integer (120794 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.01s

Solved with dual simplex

Root relaxation: objective 1.400000e+01, 11082 iterations, 1.92 seconds (0.46 work units)
Total elapsed time = 6.95s
Total elapsed time 